https://www.usatoday.com/story/tech/talkingtech/2018/02/24/7-days-fringe-mainstream-how-conspiracy-theory-ricocheted-around-web/361446002/


https://www.factcheck.org/2018/02/no-crisis-actors-parkland-florida/

In [ ]:
from json import loads

d = open("crisisactors_tweets.json").readline()
loads(d)

In [ ]:
from csv import writer

tweets = writer(open("tweets.csv","w",encoding='utf-8'))
tweets.writerow(["id","text","created_at","retweeted","in_reply_to_screen_name","in_reply_to_status_id","retweeted_screen_name","quoted_status_screen_name","source","retweet_count","screen_name","followers_count","friends_count"])

hashtags = writer(open("hashtags.csv","w",encoding='utf-8'))
hashtags.writerow(["id","created_at","screen_name","hashtag"])

urls = writer(open("urls.csv","w",encoding='utf-8'))
urls.writerow(["id","created_at","screen_name","url"])

mentions = writer(open("mentions.csv","w",encoding='utf-8'))
mentions.writerow(["id","created_at","screen_name","mention"])

media = writer(open("media.csv","w",encoding='utf-8'))
media.writerow(["id","created_at","screen_name","media"])

data = open("crisisactors_tweets.json").readlines()

for d in data:
        
    tweet = loads(d)
    
    if tweet['source']:
        ts = tweet['source']
    else: ts = ""
        
    if tweet['in_reply_to_screen_name']:
        irt = tweet['in_reply_to_screen_name']
    else: irt = ""
        
    if "retweeted_status" in tweet:
        rts = tweet["retweeted_status"]["user"]["screen_name"]
    else: rts = ""
        
    if "quoted_status" in tweet:
        qs = tweet["quoted_status"]["user"]["screen_name"]
    else: qs = ""      
   
    if tweet["in_reply_to_status_id"]:
        irtid = tweet["in_reply_to_status_id"]
    else: irtid = ""
            
    out = [tweet["id"],tweet["text"],tweet["created_at"],tweet['retweeted'],
           irt, irtid, rts, qs, ts,
           tweet["retweet_count"],tweet["user"]["screen_name"],
           tweet["user"]["followers_count"],tweet["user"]["friends_count"]]
    
    tweets.writerow(out)
    
    if 'entities' in tweet and 'hashtags' in tweet['entities']:
        save = []
        for o in tweet['entities']['hashtags']:
        
            if not o['text'] in save:
                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'],o['text']]
                hashtags.writerow(out)
                save.append(o['text'])
    
    if 'entities'in tweet and 'urls' in tweet['entities']:

        save = []
        for o in tweet['entities']['urls']:
        
            if not o['expanded_url'] in save:
                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'],o['expanded_url']]
                urls.writerow(out)
                save.append(o['expanded_url'])
                
    if 'entities'in tweet and 'user_mentions' in tweet['entities']:
        save = []        
        for o in tweet['entities']['user_mentions']:
            if not o['screen_name'] in save:
                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'],o['screen_name']]
                mentions.writerow(out)
                save.append(o['screen_name'])

    if 'entities'in tweet and 'media' in tweet['entities']:
        save = []
        for o in tweet['entities']['media']:
 
            if not o['expanded_url'] in save:
                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'],o['expanded_url']]
                media.writerow(out)
                save.append(o['expanded_url'])

In [ ]:
from pandas import to_datetime, TimeGrouper, set_option, read_csv
set_option("display.max_rows",100)
set_option("display.max_colwidth",500)

In [ ]:
from pandas import read_csv
df = read_csv("tweets.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df["stamp"] = to_datetime(df["created_at"],format='%a %b %d %H:%M:%S +0000 %Y')
df.head()

In [ ]:
df.sort_values("stamp",inplace=True)
df.head()

In [ ]:
df[df["stamp"]>= "02-20-2018"].head()

In [ ]:
df.set_index("stamp",inplace=True)
df.head()

In [ ]:
counts = df.groupby(TimeGrouper(freq='30min')).agg({"text":"count"}).rename(columns={"text":"count"})
counts.reset_index(inplace=True)

counts.head()

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("cocteautt","8YLww0QuMPVQ46meAMaq")

myplot_parts = [go.Scatter(x=counts["stamp"],y=counts["count"],mode="line")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="crisis")

In [ ]:
person = "lucianwintrich"
df[df["quoted_status_screen_name"].str.lower() == "lucianwintrich"]

In [ ]:
df["screen_name"].value_counts().head()

In [ ]:
person = "KianMcIan"
person_counts = df[df["screen_name"]==person].groupby(TimeGrouper(freq='30min')).agg({"id":"count"}).rename(columns={"id":"count"})
person_counts.reset_index(inplace=True)

person_counts.head()

In [ ]:
myplot_parts = [go.Scatter(x=person_counts["stamp"],y=person_counts["count"],mode="line")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="crisis")

In [ ]:
myplot_parts = [go.Scatter(x=counts["stamp"],y=counts["count"],mode="line",name="Total"),
                go.Scatter(x=person_counts["stamp"],y=person_counts["count"],mode="line",name="KianMcIan")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="crisis")

In [ ]:
person = df[df["screen_name"]==person]
person.head(100)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
time_window = df[(df["stamp"] >= "2018-02-21 21:00:00") & (df["stamp"] <= "2018-02-21 23:00:00")]

In [ ]:
time_window["screen_name"].value_counts().head(25)

In [ ]:
time_window_tops = time_window[time_window["screen_name"].isin(["MyFancyOne","Jeanne_vanced","browngravy_93","AmericanMom2","toksikshok","ResistOpression","UTHornsRawk","RandieK"])]

myplot_parts = [go.Scatter(x=time_window_tops["stamp"],y=time_window_tops["screen_name"],mode="markers")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="crisis")

In [ ]:
person = "UTHornsRawk"
person = df[df["screen_name"]==person]
person.head(100)

In [ ]:
retweets = df[~df["retweeted_screen_name"].isnull()]
retweet_pairs = retweets[["id","screen_name","retweeted_screen_name"]].groupby(["screen_name","retweeted_screen_name"]).agg({"id":"count"}).rename(columns={"id":"Weight"})
retweet_pairs.reset_index(inplace=True)

retweet_pairs.sort_values("count",ascending=False).head()

In [ ]:
retweet_pairs["FromType"] = "Person"
retweet_pairs["ToType"] = "Person"
retweet_pairs["Edge"] = "Retweet"
retweet_pairs.rename(columns={"screen_name":"FromName","retweeted_screen_name":"ToName"},inplace=True)
retweet_pairs.head()

In [ ]:
retweet_pairs[retweet_pairs["Weight"]>1].shape

In [ ]:
retweet_pairs[["FromType","FromName","Edge","ToType","ToName","Weight"]][retweet_pairs["Weight"]>1].to_csv("For_Graph_Commons.csv",index=False)

In [ ]:
mentions = read_csv("mentions.csv")
mentions.head()

In [ ]:
mention_pairs = mentions[["id","screen_name","mention"]].groupby(["screen_name","mention"]).agg({"id":"count"}).rename(columns={"id":"Weight"})
mention_pairs.reset_index(inplace=True)

mention_pairs.head()

In [ ]:
mention_pairs["FromType"] = "Person"
mention_pairs["ToType"] = "Person"
mention_pairs["Edge"] = "Mentioned"
mention_pairs.rename(columns={"screen_name":"FromName","mention":"ToName"},inplace=True)
mention_pairs.head()

In [ ]:
mention_pairs.sort_values("Weight",ascending=False).head()

In [ ]:
mention_pairs[["FromType","FromName","Edge","ToType","ToName","Weight"]][mention_pairs["Weight"]>1].to_csv("For_Graph_Commons2.csv",index=False)

In [ ]:
urls = read_csv("urls.csv")
urls.head()

In [ ]:
purls = urls[urls["screen_name"]=="lensjockey"]
purls

In [ ]:
urls["domain"] = urls["url"].str.replace("http.*://([^/]+).*",r"\1")
urls.head()

In [ ]:
urls["domain"].value_counts()

In [ ]:
hashtags = read_csv("hashtags.csv")
hashtags.head()

In [ ]:
hashtags["hashtag"].str.lower().value_counts()

In [ ]:
tag_pairs = hashtags[["id","screen_name","hashtag"]].groupby(["screen_name","hashtag"]).agg({"id":"count"}).rename(columns={"id":"Weight"})
tag_pairs.reset_index(inplace=True)

tag_pairs.head()

In [ ]:
tag_pairs["FromType"] = "Person"
tag_pairs["ToType"] = "Hashtag"
tag_pairs["Edge"] = "Mentioned"
tag_pairs.rename(columns={"screen_name":"FromName","hashtag":"ToName"},inplace=True)
tag_pairs.head()

In [ ]:
keep = tag_pairs["FromName"].value_counts()
keep = keep[keep>5]
keep.shape

In [347]:
tag_pairs[["FromType","FromName","Edge","ToType","ToName","Weight"]].to_csv("For_Graph_Commons3.csv",encoding="utf-8",index=False)

In [ ]:
df["in_reply_to_screen_name"].value_counts()

<pre>
2018-02-21 20:30:22
2018-02-21 20:45:24
2018-02-21 21:00:26
2018-02-21 21:15:23
2018-02-21 21:30:22
2018-02-21 21:45:20
2018-02-21 22:01:09
2018-02-21 22:15:53
2018-02-21 22:31:07
2018-02-21 22:46:07
2018-02-21 23:01:05
2018-02-21 23:15:23
2018-02-21 23:30:21
2018-02-21 23:45:22
2018-02-22 00:00:23
2018-02-22 00:15:21
2018-02-22 00:30:21
2018-02-22 00:45:21
2018-02-22 01:00:23
2018-02-22 01:15:23
2018-02-22 01:30:21
2018-02-22 01:45:22
2018-02-22 02:00:22
2018-02-22 02:15:21
</pre>